In [ ]:
#############################################################################################
######################################### LIBRARIES #########################################
#############################################################################################

from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException, NoSuchElementException
from selenium.webdriver.common.keys import Keys
from selenium.webdriver import ActionChains
from selenium.webdriver.chrome.options import Options

import os
import pandas as pd
import numpy as np

#import pymysql
import sqlite3

from datetime import datetime
from time import sleep

In [ ]:
%load_ext sql
%sql sqlite:///grocery.sqlite

In [ ]:
%%sql

---DROP TABLE smmarkets;

CREATE TABLE smmarkets (
    productID       TEXT NOT NULL,
    webpage         TEXT NOT NULL,
    productLink     TEXT NOT NULL,
    productCategory TEXT NOT NULL,
    productName     TEXT NOT NULL,
    productPrice    TEXT NOT NULL
);

In [ ]:
%%sql
SELECT * FROM smmarkets;

In [ ]:
#############################################################################################
########################################## SET UP ###########################################
#############################################################################################


### Path/Location of Chromedriver
#dirpath = os.getcwd()
filepath = 'C:/Users/Mystycalpha/chromedriver'

### File name and path/location where the dataframe will be exported to CSV
filename = "smmarkets-links.csv"
savepath = "C:/Users/Mystycalpha/Desktop/Grocery App" + filename

### List of Links
wpg_list = []
with open("smmarket-links-list-Copy.txt", 'r') as f:
        wpg_list = [l.strip("\n") for l in f]

#print(wpg_list)

### Silence Chrome
chrome_options = Options()  
chrome_options.add_argument("--headless")  

### SQL Database
conn = sqlite3.connect("grocery.sqlite") #pymysql.connect(host='127.0.0.1',user='root', passwd = '', db = 'mysql', charset = 'utf8')
cur = conn.cursor()
#cur.execute("USE groceryDB")

In [ ]:
#############################
##### Export data to DB #####

def storeDB(new_entry):
        #dateScraped = datetime.now(tz=None)
        val = (new_entry['webpage'], new_entry['productID'], new_entry['productLink'],
               new_entry['productCategory'], new_entry['productName'], new_entry['productPrice'])
        query = '''
        INSERT INTO smmarkets (webpage, productID, productLink, productCategory, productName, productPrice)
        VALUES (?, ?, ?, ?, ?, ?);
        '''                
        #print(query)
        cur.execute(query, val)
        conn.commit()
        #cur.connection.commit()

In [ ]:
##################################
##### Creating the Dataframe #####

### Make column list of categories / features for the creation of dataframe
column_list = ['webpage', 'id', 'link', 'category', 'name', 'price']

### Create empty dataframe
df = pd.DataFrame(columns=column_list)


####################################
##### Chromedriver and Browser #####

print('Path to Driver: ', filepath)

In [ ]:
#####################################
##### Run function for scraping #####


for wpg in wpg_list:
        browser = webdriver.Chrome(executable_path = filepath, options = chrome_options)
        print(wpg)
        browser.get(wpg)
        
        sleep(1)

        print("\n################################################################################")

        try:
            loaded = WebDriverWait(browser, 10).until(EC.presence_of_element_located((By.XPATH, '/html/body/div[7]/div[2]/div/div[2]')))

            category = loaded.find_element_by_xpath('/html/body/div[7]/div[2]/div/div[2]/div[2]/div[1]/div/h1').get_attribute('innerHTML')
            if "amp;" in str(category):
                category = category.replace("amp;", "")
            print('Category:', category)
            

            sleep(1)
            
            try:
                page_text = loaded.find_element_by_xpath('//*[@id="amscroll-navbar"]/span').get_attribute('innerHTML')
                pages = int(str(page_text).strip('"').split(' ')[-1])

            except Exception as e:
                print("Error occured for pages:", e)
                with open("smmarket-scrape-errors.txt", 'a+') as f:
                    f.write('%s \t Error occured for pages: \t %s\n' % (wpg, e))
                pages = 1

            print("Number of pages:", pages)
            
            with open("smmarket-scrape-errors.txt", 'a+') as f:
                f.write('%s \t Number of pages: \t %s\n' % (wpg, pages))
            
            browser.quit()

            for p in range(1, pages+1):
                wpg_pg = wpg + '?p=' + str(p)
                browser = webdriver.Chrome(executable_path = filepath, options = chrome_options)
                print(wpg_pg)
                browser.get(wpg_pg)
                
                sleep(1)

                products_grid = WebDriverWait(browser, 10).until(EC.presence_of_element_located((By.XPATH, '/html/body/div[7]/div[2]/div/div[2]/div[2]/div[2]/div[4]/ol')))
                products = products_grid.find_elements_by_class_name('product-item')
                
                sleep(1)

                for product in products:
                    ### Get ID
                    productID = "smmarkets-" + str(product.find_element_by_class_name('price-box').get_attribute('data-product-id'))

                    ### Get Price
                    price = product.find_element_by_class_name('price-wrapper').get_attribute('data-price-amount')

                    ### Get Name
                    link = product.find_element_by_class_name('product-item-link').get_attribute('href')

                    ### Get Link
                    name = product.find_element_by_class_name('product-item-link').get_attribute('innerHTML').strip()

                    ### Save into new_entry that will be appended to the dataframe
                    new_entry = {'webpage': wpg_pg.strip(), 'productID': productID, 'productLink': link, 
                                 'productCategory': category, 'productName': name, 'productPrice': price}

                    #df = df.append(new_entry, ignore_index=True)
                    storeDB(new_entry)

                    ### Print all data gathered
                    print("------------------------------------------------------------------------")
                    for label in new_entry:
                        print(label.upper(), ":", new_entry[label])
                    print("------------------------------------------------------------------------")
                    
                browser.quit()

        except Exception as e:
            print("Error occured:", e)
            
            with open("smmarket-scrape-errors.txt", 'a+') as f:
                f.write('%s \t %s \t %s\n' % (wpg, wpg_pg, e))
            
            browser.quit()

        finally:
            browser.quit()



#print("df shape:", df.shape)
#print("Finished", wpg)

In [ ]:
%%sql
SELECT COUNT(1) FROM smmarkets;

In [ ]:
%%sql
SELECT productCategory, COUNT(1) AS count
FROM smmarkets
GROUP BY productCategory;

In [ ]:
%%sql
SELECT webpage, COUNT(1) AS count
FROM smmarkets
GROUP BY webpage
ORDER BY webpage;

In [ ]:
#####################################
##### Run function for scraping #####

wpg_list = [
    ["https://shop.smmarkets.ph/index.php/snacks.html", 34, 70]
]

for w in wpg_list:
        wpg = w[0]
        start = w[1]
        pages = w[2]
        
        browser = webdriver.Chrome(executable_path = filepath, options = chrome_options)
        print(wpg)
        browser.get(wpg)
        
        sleep(1)

        print("\n################################################################################")

        try:
            loaded = WebDriverWait(browser, 10).until(EC.presence_of_element_located((By.XPATH, '/html/body/div[7]/div[2]/div/div[2]')))

            category = loaded.find_element_by_xpath('/html/body/div[7]/div[2]/div/div[2]/div[2]/div[1]/div/h1').get_attribute('innerHTML')
            if "amp;" in str(category):
                category = category.replace("amp;", "")
            print('Category:', category)
            
            sleep(1)

            print("Number of pages:", pages)
            
            #with open("smmarket-scrape-errors.txt", 'a+') as f:
            #    f.write('%s \t Number of pages: \t %s\n' % (wpg, pages))
            
            browser.quit()

            for p in range(start, pages+1):
                wpg_pg = wpg + '?p=' + str(p)
                browser = webdriver.Chrome(executable_path = filepath, options = chrome_options)
                print(wpg_pg)
                browser.get(wpg_pg)
                
                sleep(1)

                products_grid = WebDriverWait(browser, 10).until(EC.presence_of_element_located((By.XPATH, '/html/body/div[7]/div[2]/div/div[2]/div[2]/div[2]/div[4]/ol')))
                products = products_grid.find_elements_by_class_name('product-item')
                
                sleep(1)

                for product in products:
                    ### Get ID
                    productID = "smmarkets-" + str(product.find_element_by_class_name('price-box').get_attribute('data-product-id'))

                    ### Get Price
                    price = product.find_element_by_class_name('price-wrapper').get_attribute('data-price-amount')

                    ### Get Name
                    link = product.find_element_by_class_name('product-item-link').get_attribute('href')

                    ### Get Link
                    name = product.find_element_by_class_name('product-item-link').get_attribute('innerHTML').strip()

                    ### Save into new_entry that will be appended to the dataframe
                    new_entry = {'webpage': wpg_pg.strip(), 'productID': productID, 'productLink': link, 
                                 'productCategory': category, 'productName': name, 'productPrice': price}

                    #df = df.append(new_entry, ignore_index=True)
                    storeDB(new_entry)

                    ### Print all data gathered
                    print("------------------------------------------------------------------------")
                    for label in new_entry:
                        print(label.upper(), ":", new_entry[label])
                    print("------------------------------------------------------------------------")
                    
                browser.quit()

        except Exception as e:
            print("Error occured:", e)
            
            with open("smmarket-scrape-errors.txt", 'a+') as f:
                f.write('%s \t %s \t %s\n' % (wpg, wpg_pg, e))
            
            browser.quit()

        finally:
            browser.quit()



#print("df shape:", df.shape)
#print("Finished", wpg)

In [ ]:
%%sql
SELECT *
FROM smmarkets
WHERE productName LIKE "%LIGO%";

In [ ]:
df = pd.read_sql("SELECT * FROM smmarkets", conn)
df.head()

In [ ]:
df_csv = df.to_excel("smmarkets-links.xlsx")